# Text Summarizer

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


### After changing the Runtype = TPU:

In [1]:
!nvidia-smi

Sun Oct 26 11:50:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Text Summarization using HuggingFace

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 19.8 MB/s eta 0:00:00


NOTE: A ROUGE score is a metric used to evaluate the quality of machine-generated text, such as summaries or translations, by comparing it to a human-created reference text.

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transforemrs accelerate
!pip install tranformers accelerate

Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
ERROR: Could not find a version that satisfies the requirement tranformers (from versions: none)
ERROR: No matching distribution found for tranformers


In [5]:
!pip install evaluate

In [6]:
from transformers import pipeline, set_seed
import datasets
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
#from datasets import load_dataset, load_metric
import evaluate                               # first install this                        # use this instead of load_dataset

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# CUDA

A parallel computing platform and programming model created by NVIDIA that allows developers to use NVIDIA's graphics processing units (GPUs) for general-purpose processing.

We can sometimes loose the GPU instance, it can throw error and we write a lone of code to handle it.

### With runtype = CPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### With runtype = TPU

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

NOTE: Seq2Seq model for Text Summarization as it is text->text conversion model.

In [42]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [9]:
model_name = "google/pegasus-cnn_dailymail"

# Important points:

- While using HuggingFace, there is no need to do preprocessing using a pre-trained method(later).

- After getting tokens from the imported "Tokenizer", the next step is embeddings creation.

- 'Autotokenizer' is a class and all preprocessing functions are written inside it.

- All models have different tokenizers.

- No use of removing stopwords as this task is **text to text** conversion.

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)       # importing tokenizer for pegasus model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Loading pre-trained model

In [11]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Importing the dataset (from Huggingface)

In [12]:
dataset_samsum = load_dataset("knkarthick/samsum")

In [11]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [12]:
dataset_samsum["train"][1]["dialogue"]

'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [13]:
dataset_samsum["train"][1]["summary"]

'Olivia and Olivier are voting for liberals in this election. '

In [14]:
dataset_samsum["train"][500]["dialogue"]

"Ivy: Chloe just told me that you're not coming with us!\nCarter: I have a family reunion around that time\nIvy: just ditch it\nCarter: can't, not this time\nIvy: why?\nCarter: my grandfather is really sick\nCarter: might be my last chance to see him\nIvy: sorry to hear about that, didn't know\nCarter: don't worry, I'll go with all of you on the next trip\nIvy: that's not a problem, in a situation like this you should stay with your family\nCarter: that's the plan at least\nIvy: take care!"

In [15]:
dataset_samsum["train"][500]["summary"]

"Carter is not joining Ivy and Chloe due to a family reunion. Carter's grandfather is very ill."

In [16]:
for split in dataset_samsum:
  print(split)

train
validation
test


In [17]:
dataset_samsum['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14731
})

In [18]:
len(dataset_samsum['train'])

14731

In [19]:
split_lengths = [(split+":", len(dataset_samsum[split])) for split in dataset_samsum]
split_lengths

[('train:', 14731), ('validation:', 818), ('test:', 819)]

In [20]:
# Features

dataset_samsum['train'].column_names

['id', 'dialogue', 'summary']

In [21]:
dataset_samsum['train'].features

{'id': Value('string'),
 'dialogue': Value('string'),
 'summary': Value('string')}

In [22]:
dataset_samsum['train'].num_rows

14731

In [23]:
dataset_samsum['test'][1]['dialogue']

"Eric: MACHINE!\nRob: That's so gr8!\nEric: I know! And shows how Americans see Russian ;)\nRob: And it's really funny!\nEric: I know! I especially like the train part!\nRob: Hahaha! No one talks to the machine like that!\nEric: Is this his only stand-up?\nRob: Idk. I'll check.\nEric: Sure.\nRob: Turns out no! There are some of his stand-ups on youtube.\nEric: Gr8! I'll watch them now!\nRob: Me too!\nEric: MACHINE!\nRob: MACHINE!\nEric: TTYL?\nRob: Sure :)"

In [24]:
dataset_samsum['test'][1]['summary']

'Eric and Rob are going to watch a stand-up on youtube.'

In [13]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

    with tokenizer.as_target_tokenizer():
      target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True)

    return{
          'input_ids' : input_encodings['input_ids'],
          'attention_mask' : input_encodings['attention_mask'],
          'labels': target_encodings['input_ids']
    }


## Attention Mask:

- An attention mask is a binary tensor that tells a model which tokens to attend to and which to ignore, preventing it from processing irrelevant information like padding tokens or future tokens.

- It is a crucial component in Transformer models, ensuring that calculations are focused on meaningful parts of the input by setting attention scores for masked tokens to zero.  

- Suppose a model identies that bus, cat, park, human are relevant words, then it gives more attention to them(1) and makes other 0s.

Example:
```
[car, bus, cat, park, apple, horn, human, flower]

[0,    1,   1,    1,    0,     0,    1,     0]     
```

In [20]:
dataset_total = dataset_samsum.map(convert_examples_to_features, batched = True)
dataset_total

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
})

In [27]:
dataset_total['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14731
})

In [28]:
dataset_total['train']['input_ids'][10]

[12853,
 151,
 10532,
 147,
 722,
 140,
 128,
 242,
 152,
 36200,
 151,
 10532,
 186,
 147,
 36200,
 151,
 168,
 140,
 848,
 1226,
 108,
 589,
 108,
 2041,
 119,
 147,
 36200,
 151,
 125,
 188,
 419,
 8170,
 147,
 110,
 151,
 470,
 12853,
 151,
 2632,
 304,
 147,
 1509,
 990,
 147,
 12853,
 151,
 11991,
 147,
 12853,
 151,
 4385,
 114,
 924,
 148,
 112,
 129,
 5160,
 107,
 36200,
 151,
 125,
 1989,
 147,
 110,
 151,
 470,
 36200,
 151,
 18156,
 134,
 7713,
 259,
 1398,
 107,
 152,
 12853,
 151,
 7435,
 147,
 12853,
 151,
 1883,
 119,
 186,
 134,
 377,
 1638,
 152,
 36200,
 151,
 12098,
 147,
 1883,
 119,
 186,
 147,
 110,
 151,
 470,
 1]

In [29]:
dataset_total['train']['attention_mask'][10]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

## Training of the model

In [14]:
from transformers import DataCollatorForSeq2Seq                                 # to train daa in batches

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [34]:
!pip install 'accelerate>=0.26.0'

In [36]:
!pip install transformers[torch]

In [16]:
!pip install --upgrade accelerate
!pip install transformers[torch]

  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.11.0-py3-none-any.whl (375 kB)


In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', 
    num_train_epochs=1, 
    warmup_steps=500,
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,
    weight_decay=0.01, 
    logging_steps=10,
    eval_strategy='steps', 
    eval_steps=500, 
    save_steps=1e6,
    gradient_accumulation_steps=16
)

In [17]:
!pip uninstall -y accelerate transformers
!pip install --upgrade accelerate
!pip install transformers[torch]

Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.11.0-py3-none-any.whl (375 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.17.1 requires transformers, which is not installed.


  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)


In [18]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    eval_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [21]:
trainer = Trainer(model = model_pegasus, args = trainer_args,
                  tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = dataset_total['test'],
                  eval_dataset = dataset_total['validation'])

/tmp/ipython-input-2445205711.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model_pegasus, args = trainer_args,


In [23]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gadewarmonika (gadewarmonika-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=52, training_loss=2.9784007989443264, metrics={'train_runtime': 1410.5299, 'train_samples_per_second': 0.581, 'train_steps_per_second': 0.037, 'total_flos': 314017624350720.0, 'train_loss': 2.9784007989443264, 'epoch': 1.0})

# Evaluation

In [33]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    It yeild successive batch-sized chunks from list of elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i+batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size = 16, device = device,
                                column_text = "article",
                                column_summary = "highlights"):

    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total = len(article_batches)):

        inputs = tokenizer(article_batch, max_length = 1024, truncation = True,
                         padding = 'max_length', return_tensors = 'pt')

        summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                 attention_mask = inputs['attention_mask'].to(device),
                                 length_penalty = 0.8, num_beams = 8, max_length = 128)

        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the token, and add the decoded texts with the references to the metric.

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True,
                                              clean_up_tokenization_spaces = True)
                                              for s in summaries]

        metric.add_batch(predictions = decoded_summaries, references = target_batch)

        # Finally compute and return the ROUGE score
        score = metric.compute()

    return score

In [34]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = evaluate.load('rouge')

In [37]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],
    rouge_metric, trainer.model,
    tokenizer, batch_size = 2,
    column_text = 'dialogue',
    column_summary = 'summary')

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index = [f'pegasus'])

100%|██████████| 5/5 [00:30<00:00,  6.18s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.289744,0.08347,0.257692,0.257692


# Saving the trained model and tokenizer

In [38]:
model_pegasus.save_pretrained('pegasus-samsum-model')

In [39]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [44]:
# Loading the locally saved model

tokenizer = AutoTokenizer.from_pretrained('/content/tokenizer')

# Prediction

In [45]:
gen_kwargs = {'length_penalty': 0.8, "num_beams": 8, "max_length":128}

user_text = dataset_samsum['test'][0]['dialogue']
actual_summary = dataset_samsum['test'][0]['summary']

pipe = pipeline('summarization', model = 'pegasus-samsum-model', tokenizer = tokenizer)

print("Dialogue:")
print(user_text)


print("\nActual Summary:")
print(actual_summary)


print("\nModel Predicted Summary:")
print(pipe(user_text, **gen_kwargs)[0]["summary_text"])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Actual Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Predicted Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .


# Analysis:

The model is not giving accurate results maybe because the training data is relaced by test data for faster training.